In [ ]:
import time
from selenium import webdriver
from selenium.webdriver import Safari
import pandas as pd
from selenium.webdriver.common.by import By
import requests
import bs4
drive = Safari(executable_path =r"/usr/bin/safaridriver")
data = []
time.sleep(5)
for i in range(8,22,1):
    try:
        drive.get('https://www.immoweb.be/en/search/house/for-sale/brussels/district?countries=BE&maxPrice=500000&minPrice=200000'
        + '&page='+str(i)+'&orderBy=relevance')
        time.sleep(15)
        soup = bs4.BeautifulSoup(drive.page_source, "html.parser")
        jobs_elements = soup.find_all("h2", class_= "card__title card--result__title")
        for jobs_element in jobs_elements:
            try:
                table_data = jobs_element.find_all("a", class_= 'card__title-link')
                for var in table_data:
                    item = var.get('href')
            except AttributeError:
                table_data = 'no data'
            data.append(item)
        time.sleep(15)
    except:
        continue
with open('links_6.txt',"w") as f:
    for x in data:
        f.write(x+'\n')
with open('links_6.txt',"r") as r:
    array = r.read().splitlines()
    print(array)
drive.close()

In [ ]:
from time import sleep
import bs4
from selenium import webdriver
from selenium.webdriver import Safari
import pandas as pd
import threading
import numpy as np
dict = { 'ID':[], 'Locality':[],'PricePr': [], 'Tenement building':[],'Venue of the sale':[],
      'Bedrooms':[],  "Living area":[], "Kitchen type":[], "Furnished" : [] , 'How many fireplaces?':[], 'Terrace':[], 'Terrace surface':[], 
       'Garden':[], 'Garden surface':[], 'Surface of the plot':[], 'Number of frontages': [], 'Swimming pool':[], 'Building condition':[]}
data_columns = list(dict.keys())
drive = Safari(executable_path=r"/usr/bin/safaridriver")
with open("data/links_0.txt") as f:
    links = f.read().split("\n")
    for var in links:
        try:
            
            thread = threading.Thread(target=drive.get(url = var))
            soup = bs4.BeautifulSoup(drive.page_source, "html.parser") 
            table_datas = soup.find_all("tr", class_= "classified-table__row")
            thread.start()
            sleep(3)
            headers = []
            dict['ID'].append(var.rsplit('/', 1)[1])
            headers.append('ID')
            dict['Locality'].append(var.rsplit('/')[-3])
            headers.append('Locality')
            price = soup.find('p', class_='classified__price').find('span', class_='sr-only').string
            dict["PricePr"].append(price)
            headers.append('PricePr')
            for x in table_datas:
                try:
                    table_data = x.find("td", class_="classified-table__data").text.strip()
                    table_data = table_data.replace('square meters', '').replace('m²', '').replace('m', '').strip("\n").strip("\t").strip()
                except AttributeError:
                    table_data = np.NaN
                try:
                    table_header = x.find("th", class_="classified-table__header").text.strip()
                    headers.append(table_header)
                except AttributeError:
                    table_header = np.NaN
                
                if table_header == 'Tenement building':
                    dict["Tenement building"].append(table_data)
                if table_header == 'Venue of the sale':
                    dict["Venue of the sale"].append(table_data)
                if table_header == 'Living area':
                    dict["Living area"].append(table_data)
                if table_header == 'Kitchen type':
                    dict["Kitchen type"].append(table_data)
                if table_header =='Bedrooms':
                    dict["Bedrooms"].append(table_data)
                if table_header == 'Surface of the plot':
                    dict["Surface of the plot"].append(table_data)
                if table_header == 'Terrace':
                    dict["Terrace"].append(table_data)
                if table_header == 'Terrace surface':
                    dict["Terrace surface"].append(table_data)
                if table_header == 'Garden surface':
                    dict["Garden surface"].append(table_data)
                if table_header == 'Garden':
                    dict["Garden"].append(table_data)
                if table_header == 'Furnished':
                    dict["Furnished"].append(table_data)
                if table_header == 'How many fireplaces?':
                    dict["How many fireplaces?"].append(table_data)
                if table_header == 'Number of frontages':
                    dict["Number of frontages"].append(table_data)
                if table_header == 'Swimming pool':
                    dict["Swimming pool"].append(table_data)
                if table_header == 'Building condition':
                    dict["Building condition"].append(table_data)
            diff = list(set(data_columns)-set(headers))
            for y in diff:
                dict[y].append(np.NaN)
            sleep(3) 
            thread.join()
        except TimeoutError:
            print('This is TimeoutError')
        except:
            break

new_dict = pd.DataFrame(dict)
with open('page_2.csv',"w") as f:
    new_dict.to_csv('page_2.csv', index=False)
drive.close()

In [ ]:
with open('page_2.csv',"r") as f:
    df=pd.read_csv('page_2.csv')
#code to add new column bv. to fill null values with Brussel
#df['Neighbourhood or locality'] = df['Neighbourhood or locality'].fillna('Brussel')
print(df)
with open('page_2.csv',"w") as f:
    df.to_csv('page_2.csv', index=False)